In [ ]:
%tensorflow_version 1.6
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from keras.layers import Input, BatchNormalization
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

In [ ]:
print(tf.__version__)
keras.backend.image_data_format()
latent_dimension = 100

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train[:, np.newaxis, :, :]
adam = Adam(lr=0.00025, beta_1=0.7)

# Generator Model
# Transforms the input into a 7 × 7 256-channel feature map
generator = Sequential()
generator.add(Dense(256*7*7, input_dim=latent_dimension))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())
generator.add(Reshape((256, 7, 7)))
generator.add(UpSampling2D(size=(2, 2), data_format='channels_first'))
generator.add(Conv2D(64, kernel_size=(5, 5), padding='same', data_format='channels_first'))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())
generator.add(UpSampling2D(size=(2, 2), data_format='channels_first'))
generator.add(Conv2D(1, kernel_size=(5, 5), padding='same', activation='tanh', data_format='channels_first'))
print(generator.summary())
generator.compile(loss='binary_crossentropy', optimizer=adam)

# Discriminator Model
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=(5, 5), strides=(1, 1), padding='same', 
                         input_shape=(1, 28, 28), data_format='channels_first',
                         kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(BatchNormalization())
discriminator.add(Dropout(0.2))
discriminator.add(Conv2D(256, kernel_size=(5, 5), strides=(2, 2), 
                         padding='same', data_format='channels_first'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(BatchNormalization())
discriminator.add(Dropout(0.2))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
print(discriminator.summary())
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

# Adversarial Network Creation
discriminator.trainable = False
ganInput = Input(shape=(latent_dimension,))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [3]:
def plots(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, latent_dimension])
    generatedImages = generator.predict(noise)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i, 0], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('images/pics_epoch_%d.png' % epoch)

def save(epoch):
    generator.save('models/generator_epoch_%d.h5' % epoch)
    discriminator.save('models/discriminator_epoch_%d.h5' % epoch)

In [ ]:
epochs = 6
batchSize = 128
batchCount = X_train.shape[0] / batchSize

for e in range(1, epochs):
    print('Epoch %d' % e)
    for i in tqdm(range(int(batchCount))):
        # Creating input noise,images
        noise = np.random.normal(0, 1, size=[batchSize, latent_dimension])
        imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]

        # Creating wrong MNIST images
        generatedImages = generator.predict(noise)
        X = np.concatenate([imageBatch, generatedImages])

        # Labels both data
        yDis = np.zeros(2*batchSize)
        
        # One-sided label smoothing
        yDis[:batchSize] = 0.9

        # Discriminator
        discriminator.trainable = True
        dloss = discriminator.train_on_batch(X, yDis)

        # Generator
        noise = np.random.normal(0, 1, size=[batchSize, latent_dimension])
        yGen = np.ones(batchSize)
        discriminator.trainable = False
        gloss = gan.train_on_batch(noise, yGen)

    if e == 1 or e == epochs - 1:
        plots(e)
        save(e)
